In [1]:
  !pip install transformers
!pip install emoji
!pip install contractions
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 59.6 MB/s 
     |████████████████████████████████| 163 kB 87.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 14.7 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=294e1ce2d24c5bc97f96e824cb0f8c0760c2983cbf44718b7f7e1fe30951b1eb
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 15.9 MB/s 
     |████████████████████████████████| 287 kB 86.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/col

In [2]:
import contractions
import emoji
import re
from unidecode import unidecode
import re, string
from re import sub


from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
# from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Colab Notebooks/Thesis/train.csv"
df = pd.read_csv(path)

Mounted at /content/gdrive


In [4]:
def normalisation(text):

  replace_content = ''
  text = emoji.demojize(text)# transform from emoji to a string
  for k, v in contractions.contractions_dict.items():
    text = text.replace(k,v)#working word contractions
  text = re.sub(r'(.)\1+', r'\1\1', text)#removing letter repetition
  text = re.sub(r'[\?\.\!]+(?=[\?\.\!])',replace_content, text)#removing punctuation repetition
  text = re.sub('#+', ' ', text)# remove hashtag
  text = text.split()
  return text

In [5]:
#Converting sting value of the output to numeric
df = df[df.sentiment != 'neutral']
sentiment  = {'positive': 1,'negative':-1} 
df.sentiment = [sentiment[item] for item in df.sentiment] 
#removing the unimportant for our analysis column "ID" 
df.pop('id')
df.text = df["text"].str.lower()
df.text = df.text.apply(lambda x: normalisation(x))

In [6]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text']).reshape(-1, 1), np.array(df['sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'sentiment']);

In [7]:
train_os['sentiment'].value_counts()

-1    2795
 1    2795
Name: sentiment, dtype: int64

In [8]:
X = train_os['text'].values
y = train_os['sentiment'].values

In [9]:
train, test = train_test_split(train_os , test_size=0.2,  stratify=y, random_state=42)

X_test , y_test = test['text'].values, test['sentiment'].values 

X, y = train['text'].values, train['sentiment'].values 

X_train, X_valid, y_train, y_valid  = train_test_split(X, y, test_size=0.2,  stratify=y, random_state=42)

# **Simple RNN**

In [10]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [11]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [12]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(X_train) + list(X_valid)+list(X_test))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_valid)
xtest_seq = token.texts_to_sequences(X_test)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [13]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         52992600  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 53,032,801
Trainable params: 53,032,801
Non-trainable params: 0
_________________________________________________________________
CPU times: user 1.32 s, sys: 389 ms, total: 1.7 s
Wall time: 3.76 s


In [14]:
model.fit(xtrain_pad, y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/20
56/56 [==============================] - 68s 1s/step - loss: 0.0075 - accuracy: 0.0145
Epoch 2/20
56/56 [==============================] - 65s 1s/step - loss: -2.0475 - accuracy: 0.1308
Epoch 3/20
56/56 [==============================] - 64s 1s/step - loss: -0.5134 - accuracy: 0.3131
Epoch 4/20
56/56 [==============================] - 65s 1s/step - loss: -2.2506 - accuracy: 0.2899
Epoch 5/20
56/56 [==============================] - 65s 1s/step - loss: -4.8625 - accuracy: 0.3506
Epoch 6/20
56/56 [==============================] - 63s 1s/step - loss: -9.9258 - accuracy: 0.3771
Epoch 7/20
56/56 [==============================] - 64s 1s/step - loss: -14.9015 - accuracy: 0.4224
Epoch 8/20
56/56 [==============================] - 64s 1s/step - loss: -19.5151 - accuracy: 0.4375
Epoch 9/20
56/56 [==============================] - 64s 1s/step - loss: -24.1049 - accuracy: 0.4412
Epoch 10/20
56/56 [==============================] - 64s 1s/step - loss: -28.5588 - accuracy: 0.4490
Epoch 

In [15]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [16]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,y_valid)))

Auc: 0.86%


In [17]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,y_valid)})